In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [7]:
eco_df = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\teis-mikeriello\data\NSS_ECO Data Ind 3 FFY 2023-24 20241211.xlsx",sheet_name='ECO with Exit23-24 ')

In [3]:
outcome_1 = pd.read_csv('../teis-mikeriello/data/outcome1_a.csv')
outcome_2 = pd.read_csv('../teis-mikeriello/data/outcome2_a.csv')
outcome_3 = pd.read_csv('../teis-mikeriello/data/outcome3_a.csv')

pc_a = pd.concat([outcome_1, outcome_2, outcome_3], ignore_index=True)
id3 = pc_a['Student TEIDS ID']

In [45]:
new_poe_count = new_poe.value_counts('DISTRICT')

In [47]:
#new_poe = pc_a_eco = eco_df[eco_df['CHILD_ID'].isin(id3)]
new_poe_count = new_poe_count.reset_index()
new_poe_count.columns = ['DISTRICT', 'COUNT']
new_poe_count

,DISTRICT,COUNT
0,MD,33
1,GN,27
2,SC,22
3,ET,17
4,UC,12
5,FT,9
6,SE,7
7,NW,5
8,SW,5


In [49]:
mapping = {
    "FT": ("First TN", "37601"),
    "ET": ("East TN", "37920"),
    "SE": ("Southeast TN", "37406"),
    "UC": ("Upper Cumberland", "38506"),
    "GN": ("Greater Nashville", "37243"),
    "SC": ("South Central", "37130"),
    "NW": ("Northwest", "38238"),
    "SW": ("Southwest", "38301"),
    "MD": ("Memphis Delta", "38134")
}

In [51]:
new_poe_count['DISTRICT NAME'] = new_poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
new_poe_count['NEARBY CITY ZIP'] = new_poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])
new_poe_count = new_poe_count[['DISTRICT', 'DISTRICT NAME', 'NEARBY CITY ZIP', 'COUNT']]
new_poe_count

,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT
0,MD,Memphis Delta,38134,33
1,GN,Greater Nashville,37243,27
2,SC,South Central,37130,22
3,ET,East TN,37920,17
4,UC,Upper Cumberland,38506,12
5,FT,First TN,37601,9
6,SE,Southeast TN,37406,7
7,NW,Northwest,38238,5
8,SW,Southwest,38301,5


In [55]:
import pgeocode
nomi = pgeocode.Nominatim('us')

def get_coordinates(zip_code):
    location = nomi.query_postal_code(zip_code)
    return location.latitude, location.longitude

new_poe_count[['LAT', 'LON']] = new_poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))
new_poe_count

C:\Users\mrmik\AppData\Local\Temp\ipykernel_325804\3019105233.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_poe_count[['LAT', 'LON']] = new_poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))
C:\Users\mrmik\AppData\Local\Temp\ipykernel_325804\3019105233.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_poe_count[['LAT', 'LON']] = new_poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))


,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT,LAT,LON
0,MD,Memphis Delta,38134,33,35.1845,-89.8574
1,GN,Greater Nashville,37243,27,36.1650,-86.7821
2,SC,South Central,37130,22,35.8456,-86.3903
3,ET,East TN,37920,17,35.8929,-83.9387
4,UC,Upper Cumberland,38506,12,36.1819,-85.4408
5,FT,First TN,37601,9,36.3339,-82.3408
6,SE,Southeast TN,37406,7,35.0614,-85.2478
7,NW,Northwest,38238,5,36.3434,-88.8503
8,SW,Southwest,38301,5,35.6102,-88.8140


In [117]:
import folium
from folium.plugins import HeatMap, MarkerCluster

tn_map1 = folium.Map(location=[35.5175, -86.5804], zoom_start=7)

new_count_data = new_poe_count[['LAT', 'LON','COUNT']].dropna().values.tolist()

cluster = MarkerCluster(locations=locations, popups=new_poe_count['COUNT'].tolist())  
tn_map1.add_child(cluster)

HeatMap(new_count_data).add_to(tn_map1)

legend = folium.map.LayerControl(position='bottomleft', collapsed=False)
tn_map1.add_child(legend)

In [121]:
tn_map1.save("new_business_rules_poe.html")

In [141]:
import folium
from folium.plugins import HeatMap

tn_map1 = folium.Map(location=[35.5175, -86.5804], zoom_start=7)
new_count_data = new_poe_count[['LAT', 'LON', 'COUNT']].dropna().values.tolist()
HeatMap(new_count_data).add_to(tn_map1)
tn_map1

In [59]:
old_poe = pd.read_csv('../teis-mikeriello/data/old_rules_a.csv')

In [65]:
old_poe_count = old_poe.value_counts('DISTRICT')

In [67]:
old_poe_count = old_poe_count.reset_index()
old_poe_count.columns = ['DISTRICT', 'COUNT']
old_poe_count

,DISTRICT,COUNT
0,GN,359
1,MD,342
2,ET,300
3,SC,299
4,UC,203
5,FT,171
6,SE,166
7,NW,91
8,SW,90


In [69]:
old_poe_count['DISTRICT NAME'] = old_poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[0])
old_poe_count['NEARBY CITY ZIP'] = old_poe_count['DISTRICT'].map(lambda x: mapping.get(x, ('Unknown', '00000'))[1])
old_poe_count = old_poe_count[['DISTRICT', 'DISTRICT NAME', 'NEARBY CITY ZIP', 'COUNT']]
old_poe_count

,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT
0,GN,Greater Nashville,37243,359
1,MD,Memphis Delta,38134,342
2,ET,East TN,37920,300
3,SC,South Central,37130,299
4,UC,Upper Cumberland,38506,203
5,FT,First TN,37601,171
6,SE,Southeast TN,37406,166
7,NW,Northwest,38238,91
8,SW,Southwest,38301,90


In [131]:
import pgeocode
nomi = pgeocode.Nominatim('us')

def get_coordinates(zip_code):
    location = nomi.query_postal_code(zip_code)
    return location.latitude, location.longitude

old_poe_count[['LAT', 'LON']] = old_poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))
old_poe_count

C:\Users\mrmik\AppData\Local\Temp\ipykernel_325804\462425109.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_poe_count[['LAT', 'LON']] = old_poe_count['NEARBY CITY ZIP'].apply(lambda z: pd.Series(get_coordinates(z)))


,DISTRICT,DISTRICT NAME,NEARBY CITY ZIP,COUNT,LAT,LON
0,GN,Greater Nashville,37243,359,36.1650,-86.7821
1,MD,Memphis Delta,38134,342,35.1845,-89.8574
2,ET,East TN,37920,300,35.8929,-83.9387
3,SC,South Central,37130,299,35.8456,-86.3903
4,UC,Upper Cumberland,38506,203,36.1819,-85.4408
5,FT,First TN,37601,171,36.3339,-82.3408
6,SE,Southeast TN,37406,166,35.0614,-85.2478
7,NW,Northwest,38238,91,36.3434,-88.8503
8,SW,Southwest,38301,90,35.6102,-88.8140


In [125]:
import folium
from folium.plugins import HeatMap, MarkerCluster

tn_map2 = folium.Map(location=[35.5175, -86.5804], zoom_start=7)

old_count_data = old_poe_count[['LAT', 'LON', 'COUNT']].dropna().values.tolist()

cluster = MarkerCluster(locations=locations, popups=old_poe_count['COUNT'].tolist())  
tn_map2.add_child(cluster)

HeatMap(old_count_data).add_to(tn_map2)

legend = folium.map.LayerControl(position='bottomleft', collapsed=False)
tn_map2.add_child(legend)

In [127]:
tn_map2.save("old_business_rules_poe.html")

In [145]:
import folium
from folium.plugins import HeatMap

tn_map2 = folium.Map(location=[35.5175, -86.5804], zoom_start=7)
old_count_data = old_poe_count[['LAT', 'LON', 'COUNT']].dropna().values.tolist()
HeatMap(old_count_data).add_to(tn_map2)
tn_map2